I. Querying the SQL database

II. Calculating summary statistics

III. Getting the weather data from the DarkSky API

IV. Loading the data into MongoDB

I. SQL database

In [1]:
import pandas as pd

import sqlite3 

conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

Inspecting the first table, 'matches'

In [2]:
cur.execute("""SELECT * 
                FROM Matches;""")

df1 = pd.DataFrame(cur.fetchall())

df1.columns = [x[0] for x in cur.description]
df1.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


Table: Matches Match_ID (int): unique ID per match Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League) Season (int): Season the match took place in (usually covering the period of August till May of the following year) Date (str): Date of the match HomeTeam (str): Name of the home team AwayTeam (str): Name of the away team FTHG (int) (Full Time Home Goals): Number of goals scored by the home team FTAG (int) (Full Time Away Goals): Number of goals scored by the away team FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

Selecting all the information from the matches table during the 2011 season for teams in the Bundesliga only

In [3]:
cur.execute("""SELECT * 
                FROM Matches
                WHERE season = '2011' AND Div != 'E0';""")

df2 = pd.DataFrame(cur.fetchall())

df2.columns = [x[0] for x in cur.description]
df2.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


Selecting individual dates from which to extract weather information based on dates on which bundesliga teams played.

In [4]:
cur.execute("""SELECT DISTINCT Date
                FROM Matches
                WHERE Season = '2011' AND Div != 'E0'
                ORDER BY Date ASC;""")

dates = pd.DataFrame(cur.fetchall())

dates.columns = [x[0] for x in cur.description]
dates.head()

,Date
0,2011-07-15
1,2011-07-16
2,2011-07-17
3,2011-07-18
4,2011-07-22


Selecting relevant data (match_ID, season, date of match, home team, away team, home team goals, away team goals and information about the match) from the matches table during the 2011 season for the bundesliga.

In [5]:
cur.execute("""SELECT Match_ID, Div, Season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR
            FROM Matches
            WHERE season = '2011' AND Div != 'E0';""")

df3 = pd.DataFrame(cur.fetchall())

df3.columns = [x[0] for x in cur.description]
df3.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


Investigating the number of matches where the home team won.

In [6]:
cur.execute("""SELECT DISTINCT HomeTeam, Match_ID, Div, Season, Date, FTHG, FTR
            FROM Matches
            WHERE season = '2011' AND FTR = 'H' AND Div != 'E0'
            GROUP BY HomeTeam;""")

home_wins = pd.DataFrame(cur.fetchall())

home_wins.columns = [x[0] for x in cur.description]
home_wins.head()


,HomeTeam,Match_ID,Div,Season,Date,FTHG,FTR
0,Aachen,1247,D2,2011,2012-04-29,1,H
1,Augsburg,1189,D1,2011,2012-03-31,2,H
2,Bayern Munich,1178,D1,2011,2011-12-16,3,H
3,Bochum,1154,D2,2011,2011-09-25,2,H
4,Braunschweig,1347,D2,2011,2011-11-27,4,H


Checking the number of wins for the home team in 2011

In [7]:
cur.execute("""SELECT HomeTeam as HOMETEAM, COUNT(FTR) as home_wins
            FROM Matches
            WHERE season = '2011' AND FTR = 'H' AND Div != 'E0'
            GROUP BY HomeTeam;""")

home_wins = pd.DataFrame(cur.fetchall())

home_wins.columns = [x[0] for x in cur.description]
home_wins

,HOMETEAM,home_wins
0,Aachen,4
1,Augsburg,6
2,Bayern Munich,14
3,Bochum,7
4,Braunschweig,6
5,Cottbus,4
6,Dortmund,14
7,Dresden,8
8,Duisburg,8
9,Ein Frankfurt,11


Checking the number of losses for the home team in 2011

In [8]:
cur.execute("""SELECT HomeTeam as hometeam, COUNT(FTR) as home_losses
            FROM Matches
            WHERE season = '2011' AND FTR = 'A' AND Div != 'E0'
            GROUP BY HomeTeam;""")

home_losses = pd.DataFrame(cur.fetchall())

home_losses.columns = [x[0] for x in cur.description]
home_losses

,hometeam,home_losses
0,Aachen,7
1,Augsburg,4
2,Bayern Munich,2
3,Bochum,7
4,Braunschweig,3
5,Cottbus,5
6,Dortmund,1
7,Dresden,4
8,Duisburg,7
9,Ein Frankfurt,1


There is one row missing in the home_losses table: Hannover does not appear in it. This means Hannover did not suffer any losses when it played as a home team. 

We can amend the home_wins query, remove the entry for Hannover, and then reinsert the data from Hannover after we have merged the two tables for home team wins and home team losses.

In [9]:
cur.execute("""SELECT HomeTeam as HOMETEAM, COUNT(FTR) as home_wins
            FROM Matches
            WHERE season = '2011' AND FTR = 'H' AND Div != 'E0' AND HomeTeam != 'Hannover'
            GROUP BY HomeTeam;""")

home_wins2 = pd.DataFrame(cur.fetchall())

home_wins2.columns = [x[0] for x in cur.description]
home_wins2

,HOMETEAM,home_wins
0,Aachen,4
1,Augsburg,6
2,Bayern Munich,14
3,Bochum,7
4,Braunschweig,6
5,Cottbus,4
6,Dortmund,14
7,Dresden,8
8,Duisburg,8
9,Ein Frankfurt,11


In [10]:
home_wins_losses = pd.concat([home_wins2,home_losses], axis = 1)

In [11]:
home_wins_losses

,HOMETEAM,home_wins,hometeam,home_losses
0,Aachen,4,Aachen,7
1,Augsburg,6,Augsburg,4
2,Bayern Munich,14,Bayern Munich,2
3,Bochum,7,Bochum,7
4,Braunschweig,6,Braunschweig,3
5,Cottbus,4,Cottbus,5
6,Dortmund,14,Dortmund,1
7,Dresden,8,Dresden,4
8,Duisburg,8,Duisburg,7
9,Ein Frankfurt,11,Ein Frankfurt,1


In [12]:
Hannover = pd.DataFrame({"HOMETEAM":['Hannover'], 
                    "home_wins":[10], "hometeam":['Hannover'], "home_losses":[0]})

In [13]:
Home_Wins_Losses = home_wins_losses.append(Hannover, ignore_index = True)

Checking the number of wins for the away teams in 2011

In [14]:
cur.execute("""SELECT AwayTeam as AWAYTEAM, COUNT(FTR) as away_wins
            FROM Matches
            WHERE season = '2011' AND FTR = 'A' AND Div != 'E0'
            GROUP BY AwayTeam;""")

away_wins = pd.DataFrame(cur.fetchall())

away_wins.columns = [x[0] for x in cur.description]
away_wins

,AWAYTEAM,away_wins
0,Aachen,2
1,Augsburg,2
2,Bayern Munich,9
3,Bochum,3
4,Braunschweig,4
5,Cottbus,4
6,Dortmund,11
7,Dresden,4
8,Duisburg,2
9,Ein Frankfurt,9


In [15]:
cur.execute("""SELECT AwayTeam as awayteam, COUNT(FTR) as away_losses
            FROM Matches
            WHERE season = '2011' AND FTR = 'H' AND Div != 'E0'
            GROUP BY AwayTeam;""")

away_losses = pd.DataFrame(cur.fetchall())

away_losses.columns = [x[0] for x in cur.description]
away_losses

,awayteam,away_losses
0,Aachen,8
1,Augsburg,8
2,Bayern Munich,5
3,Bochum,10
4,Braunschweig,6
5,Cottbus,10
6,Dortmund,2
7,Dresden,9
8,Duisburg,8
9,Ein Frankfurt,5


In order for our data sets to match, we must delete the entries from Hannover above and append them to our new dataframe.

In [16]:
cur.execute("""SELECT AwayTeam as AWAYTEAM, COUNT(FTR) as away_wins
            FROM Matches
            WHERE season = '2011' AND FTR = 'A' AND Div != 'E0' AND AwayTeam != 'Hannover'
            GROUP BY AwayTeam;""")

away_wins2 = pd.DataFrame(cur.fetchall())

away_wins2.columns = [x[0] for x in cur.description]
away_wins2

,AWAYTEAM,away_wins
0,Aachen,2
1,Augsburg,2
2,Bayern Munich,9
3,Bochum,3
4,Braunschweig,4
5,Cottbus,4
6,Dortmund,11
7,Dresden,4
8,Duisburg,2
9,Ein Frankfurt,9


In [17]:
cur.execute("""SELECT AwayTeam as awayteam, COUNT(FTR) as away_losses
            FROM Matches
            WHERE season = '2011' AND FTR = 'H' AND Div != 'E0' AND awayteam != 'Hannover'
            GROUP BY AwayTeam;""")

away_losses2 = pd.DataFrame(cur.fetchall())

away_losses2.columns = [x[0] for x in cur.description]
away_losses2

,awayteam,away_losses
0,Aachen,8
1,Augsburg,8
2,Bayern Munich,5
3,Bochum,10
4,Braunschweig,6
5,Cottbus,10
6,Dortmund,2
7,Dresden,9
8,Duisburg,8
9,Ein Frankfurt,5


In [18]:
away_wins_losses = pd.concat([away_wins2,away_losses2], axis = 1)

In [19]:
away_wins_losses

,AWAYTEAM,away_wins,awayteam,away_losses
0,Aachen,2,Aachen,8
1,Augsburg,2,Augsburg,8
2,Bayern Munich,9,Bayern Munich,5
3,Bochum,3,Bochum,10
4,Braunschweig,4,Braunschweig,6
5,Cottbus,4,Cottbus,10
6,Dortmund,11,Dortmund,2
7,Dresden,4,Dresden,9
8,Duisburg,2,Duisburg,8
9,Ein Frankfurt,9,Ein Frankfurt,5


In [20]:
Hannover = pd.DataFrame({"AWAYTEAM":['Hannover'], 
                    "away_wins":[2], "awayteam":['Hannover'], "away_losses":[10]})

In [21]:
Away_Wins_Losses = away_wins_losses.append(Hannover, ignore_index = True)

In [22]:
total_wins_losses = pd.concat([Home_Wins_Losses,Away_Wins_Losses], axis = 1)

In [23]:
total_wins_losses

,HOMETEAM,home_wins,hometeam,home_losses,AWAYTEAM,away_wins,awayteam,away_losses
0,Aachen,4,Aachen,7,Aachen,2,Aachen,8
1,Augsburg,6,Augsburg,4,Augsburg,2,Augsburg,8
2,Bayern Munich,14,Bayern Munich,2,Bayern Munich,9,Bayern Munich,5
3,Bochum,7,Bochum,7,Bochum,3,Bochum,10
4,Braunschweig,6,Braunschweig,3,Braunschweig,4,Braunschweig,6
5,Cottbus,4,Cottbus,5,Cottbus,4,Cottbus,10
6,Dortmund,14,Dortmund,1,Dortmund,11,Dortmund,2
7,Dresden,8,Dresden,4,Dresden,4,Dresden,9
8,Duisburg,8,Duisburg,7,Duisburg,2,Duisburg,8
9,Ein Frankfurt,11,Ein Frankfurt,1,Ein Frankfurt,9,Ein Frankfurt,5


In [28]:
total_wins_losses['total_wins_overall'] = total_wins_losses['home_wins'] + total_wins_losses['away_wins']

In [29]:
total_wins_losses

,HOMETEAM,home_wins,hometeam,home_losses,AWAYTEAM,away_wins,awayteam,away_losses,total_wins_overall
0,Aachen,4,Aachen,7,Aachen,2,Aachen,8,6
1,Augsburg,6,Augsburg,4,Augsburg,2,Augsburg,8,8
2,Bayern Munich,14,Bayern Munich,2,Bayern Munich,9,Bayern Munich,5,23
3,Bochum,7,Bochum,7,Bochum,3,Bochum,10,10
4,Braunschweig,6,Braunschweig,3,Braunschweig,4,Braunschweig,6,10
5,Cottbus,4,Cottbus,5,Cottbus,4,Cottbus,10,8
6,Dortmund,14,Dortmund,1,Dortmund,11,Dortmund,2,25
7,Dresden,8,Dresden,4,Dresden,4,Dresden,9,12
8,Duisburg,8,Duisburg,7,Duisburg,2,Duisburg,8,10
9,Ein Frankfurt,11,Ein Frankfurt,1,Ein Frankfurt,9,Ein Frankfurt,5,20


In [32]:
total_wins_losses['total_losses_overall'] = total_wins_losses['home_losses'] + total_wins_losses['away_losses']

In [33]:
total_wins_losses

,HOMETEAM,home_wins,hometeam,home_losses,AWAYTEAM,away_wins,awayteam,away_losses,total_wins_overall,total_losses_overall
0,Aachen,4,Aachen,7,Aachen,2,Aachen,8,6,15
1,Augsburg,6,Augsburg,4,Augsburg,2,Augsburg,8,8,12
2,Bayern Munich,14,Bayern Munich,2,Bayern Munich,9,Bayern Munich,5,23,7
3,Bochum,7,Bochum,7,Bochum,3,Bochum,10,10,17
4,Braunschweig,6,Braunschweig,3,Braunschweig,4,Braunschweig,6,10,9
5,Cottbus,4,Cottbus,5,Cottbus,4,Cottbus,10,8,15
6,Dortmund,14,Dortmund,1,Dortmund,11,Dortmund,2,25,3
7,Dresden,8,Dresden,4,Dresden,4,Dresden,9,12,13
8,Duisburg,8,Duisburg,7,Duisburg,2,Duisburg,8,10,15
9,Ein Frankfurt,11,Ein Frankfurt,1,Ein Frankfurt,9,Ein Frankfurt,5,20,6
